# Agrupar glosas de diagnosticos similares

- En este notebook se quiere agrupar la glosa de diagnosticos luego de haberlos preprocesados
(segun la funcion que esta en src/data/build_model.py). Esto, ya que existe redundancia en las glosas
(Por ejemplo: 4 glosas distintas para la Tetralogia de Fallot).

- Para esto se utilizara K-Means, y se iterara en la cantidad de clusters a generar. Esta informacion
se podria obtener del equipo de Patologias Congenitas.